# Reporte de problemas fitosanitarios en plantaciones de agave
--------------------

## Equipo 36

| Nombre | Matrícula |
| ------ | --------- |
| André Martins Cordebello | A00572928 |
| Enrique Eduardo Solís Da Costa | A00572678 |
| Delbert Francisco Custodio Vargas | A01795613 |

## Avance 2: Feature engineering

- Crear nuevas características para mejorar el rendimiento de los modelos.
- Mitigar el riesgo de características sesgadas y acelerar la convergencia de algunos algoritmos.

# Feature Engineering

## EDA

Durante el avance 1 fue posible comprender ciertos comportamientos de nuestro dataset, así como la selección de columnas que sí aportaban información para entender de mejor manera las características de infección de gorgojos del agave en predios del mismo.

Con esto, usando como base los datasets compartidos por la CNIT fue posible obtener el dataset llamada `all_historic_captures.xlsx`, el cual contiene información del 2014 a Agosto de 2025 sobre el nivel de incidencia o infección encontrado  en predios de agave. Estas muestras se obtuvieron con base a las condiciones definidas en el **Manual Operativo de la campaña contra plagas reglamentadas del Agave**, disponible en https://www.gob.mx/cms/uploads/attachment/file/234136/Manual_Operativo_de_la_campa_a_contra_plagas_reglamentadas_del_agave_2017.compressed.pdf, lo cual fue confirmado por nuestro Sponsor (CNIT).

Por lo anterior, el objetivo principal de este proyecto integrador es el desarrollo de un ChatBot el cual:

- Pueda recibir reportes que incluyan ubicación, breve descripción de lo encontrado, fotografías y clasificación de riesgo de parte del cuerpo técnico y ciudadanía en general.
- Este ChatBot también debe tener la capacidad de responder y alertar a los usuarios sobre focos de infección reportados o confirmados en las cercanías.

La recepción de esta información es importante ya que permitirá tomar decisiones sobre lo que las brigadas de desinfección deben atacar primero.

Por lo anterior, durante nuestro EDA (análisis exploratorio) encontramos los siguientes hallazgos:

- La captura de gorgojos del agave en las trampas tiende a aumentar en épocas lluviosas.
- Los focos de infección severos y moderados tienen una dispersión geográfica menor.
- No es normal encontrar predios de más de 8 años de antiguedad, lo que nos da una idea de cuáles predios podrían estar posiblemente abandonados.
- La mayor densidad de trampas se encuentra en el estado de Jalisco.
- No es normal encontrar focos severos de infección, pero la presencia de éstos aumenta en la época lluviosa. Es posible confirmar que los casos severos de infección son casos atípicos, ya que el valor del percentil 95 se encuentra en 17.5 capturas por trampa.
- Durante la pandemia (2020 a 2024 aproximadamente), el muestreo de las trampas colocados no fue tan constante como en años posteriores o previos a la pandemia. Esto causa un efecto de sesgo en nuestro dataset.


Y al transformar un poco nuestro dataset obtuvimos estas columnas finales:

| Feature  | Tipo | Notas |
| -------  | ---- | ----- |
| tramp_id | object | Es el ID único que se le da a la trampa al colocarse en alguna parcela o predio. Una misma trampa puede colocarse en distintos predios, pero la identificación de la misma cambia acorde a dónde se colocó. |
| sampling_date | datetime64[ns] | Es la fecha en la que se llevó a cabo el conteo de cadáveres de gorgojo en la trampa.|
| lat y lon | float64 | La `latitud` y `longitud` permiten conocer la ubicación de donde se llevó a cabo el muestreo. |
| municipality | object | Municipio al que pertenecía la trampa al momento de hacer el muestreo. |
| square_area | float64 | Area que cubre el predio donde se colocó la trampa. |
| plantation_age | float64 | Años que una plantación de agave tiene desde la última purga. |
| capture_count | float64 | Cantidad de gorgojos del agave encontrados dentro de la trampa. |
| state | object | Estado de México donde se encontraba la trampa colocada. |
| severity | object | lLa severidad de la infestación encontrada durante el muestreo. Estos niveles fueron definidos por la SICAFI.|
| Month | int32 | Mes en que se llevó a cabo el muestreo de la trampa. |
| Year | int32 | Año en que se llevó a cabo el muestreo de la trampa. |
| MonthName | object | Nombre del   mes en que se llevó a cabo el muestreo de la trampa. |
| MonthYear | datetime64[ns] |  Combinación del año y mes en que se llevó a cabo el muestreo de la trampa. |

Con esto, estaremos trabajando la ingeniería de característiscas con base en estas columnas.

## Inicio de FE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os


plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [6]:
# Carga de dataset

all_historic_captures_df = pd.read_excel( "all_historic_captures.xlsx", sheet_name="historic_captures", header= 0)
all_historic_captures_df.head(5)

,tramp_id,sampling_date,lat,lon,municipality,square_area,plantation_age,capture_count,state,severity,Month,Year,MonthName,MonthYear,plantation_age_group,surface_group,no_area
0,146_THUE13-11-023-0637T01,2022-07-21,20.401984,-101.702729,PÉNJAMO,1.75,0,8.0,GUANAJUATO,1-25,7,2022,Jul,2022-07-01,0,1-2,False
1,146_THUE13-11-023-0637T01,2022-04-19,20.401984,-101.702729,PÉNJAMO,1.75,0,5.0,GUANAJUATO,1-25,4,2022,Apr,2022-04-01,0,1-2,False
2,146_THUE13-11-023-0637T01,2022-02-25,20.401984,-101.702729,PÉNJAMO,1.75,0,5.0,GUANAJUATO,1-25,2,2022,Feb,2022-02-01,0,1-2,False
3,146_THUE13-11-023-0637T01,2022-02-14,20.401984,-101.702729,PÉNJAMO,1.75,0,4.0,GUANAJUATO,1-25,2,2022,Feb,2022-02-01,0,1-2,False
4,146_THUE13-11-023-0637T01,2022-05-18,20.401984,-101.702729,PÉNJAMO,1.75,0,6.0,GUANAJUATO,1-25,5,2022,May,2022-05-01,0,1-2,False


In [7]:
all_historic_captures_df.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
lat,827856.0,20.642391,0.728798,18.987685,20.161786,20.597498,20.884595,23.439659
lon,827856.0,-102.755748,1.301210,-114.108837,-103.721093,-102.650350,-102.184024,-98.662612
square_area,827856.0,5.020235,13.905933,0.000000,0.000000,0.510000,3.935500,787.820000
plantation_age,827856.0,2.428498,1.825808,0.000000,1.000000,2.000000,4.000000,19.000000
capture_count,827856.0,4.264352,8.156295,0.000000,0.000000,2.000000,5.000000,427.000000
Month,827856.0,7.053050,3.359216,1.000000,4.000000,7.000000,10.000000,12.000000
Year,827856.0,2018.446555,3.652609,2014.000000,2016.000000,2017.000000,2021.000000,2025.000000
plantation_age_group,827856.0,2.428498,1.825808,0.000000,1.000000,2.000000,4.000000,19.000000


In [8]:
all_historic_captures_df.describe(exclude='number').T

,count,unique,top,freq,mean,min,25%,50%,75%,max
tramp_id,827856.0,84063.0,716.0,125.0,NaN,NaN,NaN,NaN,NaN,NaN
sampling_date,827856,NaN,NaN,NaN,2018-12-29 19:19:48.102277888,2014-01-01 00:00:00,2016-05-19 00:00:00,2017-11-10 00:00:00,2021-12-28 00:00:00,2025-08-31 00:00:00
municipality,827856,149,ARANDAS,61191,NaN,NaN,NaN,NaN,NaN,NaN
state,827856,6,JALISCO,442628,NaN,NaN,NaN,NaN,NaN,NaN
severity,827856,4,1-25,518659,NaN,NaN,NaN,NaN,NaN,NaN
MonthName,827856,12,Oct,86328,NaN,NaN,NaN,NaN,NaN,NaN
MonthYear,827856,NaN,NaN,NaN,2018-12-13 13:38:05.301792,2014-01-01 00:00:00,2016-05-01 00:00:00,2017-11-01 00:00:00,2021-12-01 00:00:00,2025-08-01 00:00:00
surface_group,428434,6,2-5,143499,NaN,NaN,NaN,NaN,NaN,NaN
no_area,827856,2,False,428434,NaN,NaN,NaN,NaN,NaN,NaN
